In [2]:
import sopa
import spatialdata
import pandas as pd

/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


In [3]:
sdata = spatialdata.read_zarr("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V5.zarr")
sdata

version mismatch: detected: RasterFormatV02, requested: FormatV04
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/zarr/creati

SpatialData object, with associated Zarr store: /Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V5.zarr
├── Images
│     ├── 'HE': DataTree[cyx] (3, 6926, 21997), (3, 3464, 10999), (3, 1731, 5500), (3, 866, 2750), (3, 433, 1375), (3, 216, 687)
│     ├── 'PhenoCycler_Image': DataTree[cyx] (25, 6391, 20981), (25, 3195, 10490), (25, 1597, 5245), (25, 798, 2622), (25, 399, 1311)
│     └── 'Xenium_Image': DataTree[cyx] (5, 17141, 51168), (5, 8570, 25584), (5, 4285, 12792), (5, 2142, 6396), (5, 1071, 3198)
├── Labels
│     ├── 'cell_labels': DataTree[yx] (17141, 51168), (8570, 25584), (4285, 12792), (2142, 6396), (1071, 3198)
│     └── 'nucleus_labels': DataTree[yx] (17141, 51168), (8570, 25584), (4285, 12792), (2142, 6396), (1071, 3198)
├── Points
│     ├── 'XeniumLnmk': DataFrame with shape: (<Delayed>, 2) (2D points)
│     ├── 'Xenium_Lnmks': DataFrame with shape: (<Delayed>, 2) (2D points)
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 13) (3D points)
├── S

In [6]:
del sdata.points["Xenium_Lnmks"]
del sdata.points["XeniumLnmk"]

KeyError: 'Xenium_Lnmks'

# Phenocycler Data will be introduced as obs column !

In [7]:
adata = sdata.tables["GeneTranscripts_XStock"]
adata


AnnData object with n_obs × n_vars = 113273 × 5101
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'cell_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'

In [8]:
adata_b = sdata.tables["GeneTranscripts_XStock"]
adata_a = sdata.tables["Proteins_PhenoCycler_byXStock"]

#for idx in range(adata_b.obs.shape[1]):
#    adata_b.obs[str(idx)] = adata_a.X[:,idx]

import scipy.sparse as sp

# 1) Your provided mapping from index to channel name
channel_names = {
    '0':  'DAPI_PhenCyc', 
    '1':  'CD8_PhenCyc', 
    '2':  'CD31_PhenCyc', 
    '3':  'CD11c_PhenCyc', 
    '4':  'CD34_PhenCyc', 
    '5':  'CD20_PhenCyc', 
    '6':  'CD4_PhenCyc', 
    '7':  'CD21_PhenCyc', 
    '8':  'Ki67_PhenCyc', 
    '9':  'CD68_PhenCyc', 
    '10': 'E_cadherin_PhenCyc', 
    '11': 'pten_PhenCyc', 
    '12': 'NF-H_PhenCyc', 
    '13': 'FOXp3_PhenCyc', 
    '14': 'CD45_PhenCyc', 
    '15': 'TFAM_PhenCyc', 
    '16': 'PDL-1_PhenCyc', 
    '17': 'SOX2_PhenCyc', 
    '18': 'Vimentin_PhenCyc', 
    '19': 'HIF1A_PhenCyc', 
    '20': 'CD44_PhenCyc', 
    '21': 'Keratin_8_18_PhenCyc', 
    '22': 'aSMA_PhenCyc', 
    '23': 'EpCAM_PhenCyc', 
    '24': 'K14_PhenCyc'
}

# 2) Extract a dense array from adata_a.X if necessary
X = adata_a.X.toarray() if sp.issparse(adata_a.X) else adata_a.X

# 4) Loop over each column and assign it using the channel_names dict
for idx in range(X.shape[1]):
    col_name = channel_names[str(idx)]
    adata_b.obs[col_name] = X[:, idx]

In [9]:
adata = adata_b
adata

AnnData object with n_obs × n_vars = 113273 × 5101
    obs: 'cell_id', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'region', 'z_level', 'cell_labels', 'DAPI_PhenCyc', 'CD8_PhenCyc', 'CD31_PhenCyc', 'CD11c_PhenCyc', 'CD34_PhenCyc', 'CD20_PhenCyc', 'CD4_PhenCyc', 'CD21_PhenCyc', 'Ki67_PhenCyc', 'CD68_PhenCyc', 'E_cadherin_PhenCyc', 'pten_PhenCyc', 'NF-H_PhenCyc', 'FOXp3_PhenCyc', 'CD45_PhenCyc', 'TFAM_PhenCyc', 'PDL-1_PhenCyc', 'SOX2_PhenCyc', 'Vimentin_PhenCyc', 'HIF1A_PhenCyc', 'CD44_PhenCyc', 'Keratin_8_18_PhenCyc', 'aSMA_PhenCyc', 'EpCAM_PhenCyc', 'K14_PhenCyc'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatialdata_attrs'
    obsm: 'spatial'

# After That Point the calculated skimage will be introduced as obs to anndata

In [10]:
#Read all of the properties CSVs and collect into one merged DataFrame

import pandas as pd

# 1) Read all your CSVs into a list of DataFrames
dfs = []
for i in range(4):
    path = f"/Volumes/DATA_2025/Projects/RAO/scripts/skimage_props_feature_extraction/Region_1_ExtractedProps_forChannel{i}.csv"
    # assume your files have a header row and the first column is an index
    df = pd.read_csv(path, index_col=0)
    dfs.append(df)

# 2) Make sure they all have the same shape, index & columns
assert all(df.shape == dfs[0].shape for df in dfs)
assert all((df.index == dfs[0].index).all()   for df in dfs)
assert all((df.columns == dfs[0].columns).all() for df in dfs)

# 3) Create the merged DataFrame of object‐dtype
merged = dfs[0].copy().astype(object)

# 4) Fill each cell with a list of values from each df
for idx in merged.index:
    for col in merged.columns:
        merged.at[idx, col] = [df.at[idx, col] for df in dfs]

In [11]:
from collections import Counter

# 4b) After you’ve built each list, check for a majority
for idx in merged.index:
    for col in merged.columns:
        vals = [df.at[idx, col] for df in dfs]  # your list of 4 values
        cnt = Counter(vals)
        # find the most common value and its count
        most_common_val, count = cnt.most_common(1)[0]
        if count > len(vals) / 2:
            # majority found → replace with the single value
            merged.at[idx, col] = most_common_val
        else:
            # no majority → keep the full list
            merged.at[idx, col] = vals

In [12]:
required_columns = ['label', 'area', "centroid-0","centroid-1","centroid_local-1", "centroid_weighted_local-0", "centroid_weighted_local-1"  ,"euler_number","extent","feret_diameter_max","intensity_max","intensity_min","intensity_mean","solidity",'eccentricity', 'perimeter', "equivalent_diameter_area"]
props_df_req = merged[required_columns].iloc[:, 1:]
#props_df_req = props_df_req.rename(columns=lambda c: f"0_{c}")
props_df_req.index = props_df_req.index.astype(str)

#obs_props_df_req =  pd.concat([adata.obs, props_df_req], axis=1)
obs_props_df_req = adata.obs.join(props_df_req)
obs_props_df_req

,cell_id,transcript_counts,control_probe_counts,genomic_control_counts,control_codeword_counts,unassigned_codeword_counts,deprecated_codeword_counts,total_counts,cell_area,nucleus_area,...,euler_number,extent,feret_diameter_max,intensity_max,intensity_min,intensity_mean,solidity,eccentricity,perimeter,equivalent_diameter_area
0,aaaacnhm-1,247,0,0,0,0,0,247,57.709690,36.125001,...,1,0.632963,49.396356,"[4513.0, 5608.0, 2065.0, 10627.0]","[216.0, 981.0, 357.0, 1466.0]","[2339.9081469648563, 1993.5, 939.4944089456868...",0.949924,0.741747,138.267027,39.926131
1,aaaahgjd-1,78,0,0,0,0,0,78,65.341096,28.087189,...,1,0.753191,47.67599,"[4291.0, 3077.0, 2096.0, 6483.0]","[108.0, 295.0, 219.0, 316.0]","[1775.0734463276835, 1118.906779661017, 768.68...",0.956111,0.396934,144.225397,42.460655
2,aaabfkgb-1,282,0,0,0,0,0,282,52.877971,30.074064,...,1,0.740385,41.761226,"[4410.0, 4362.0, 2040.0, 10802.0]","[206.0, 820.0, 292.0, 598.0]","[2342.1246753246755, 1966.31341991342, 1047.37...",0.967337,0.334369,128.225397,38.348294
3,aaabhnga-1,377,0,0,0,0,0,377,54.955158,34.273595,...,1,0.721429,43.174066,"[4106.0, 5014.0, 2674.0, 6450.0]","[59.0, 178.0, 59.0, 494.0]","[1741.4158415841584, 1905.1344884488449, 1006....",0.971154,0.409994,128.953319,39.283156
4,aaacamck-1,155,0,0,0,0,0,155,37.524845,16.165938,...,1,0.669124,38.626416,"[4372.0, 4702.0, 2295.0, 10920.0]","[156.0, 545.0, 237.0, 838.0]","[1848.235006119951, 2061.341493268054, 988.511...",0.953326,0.668938,110.225397,32.252701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113268,oiinfddn-1,13,0,0,0,0,0,13,31.112657,21.223438,...,1,0.6537,42.201896,"[3626.0, 4292.0, 906.0, 787.0]","[91.0, 254.0, 46.0, 37.0]","[1824.3962264150944, 858.3788098693759, 236.42...",0.927322,0.765625,109.497475,29.61861
113269,oiinhcgf-1,299,0,0,0,0,0,299,104.762504,10.024688,...,1,0.737476,58.309519,"[633.0, 1900.0, 2688.0, 10963.0]","[33.0, 114.0, 19.0, 43.0]","[178.1080723423026, 461.1636524040582, 559.082...",0.96715,0.368353,182.267027,53.725544
113270,oiioeenn-1,23,0,0,0,0,0,23,11.514844,11.514844,...,1,0.708333,22.825424,"[4545.0, 1008.0, 345.0, 723.0]","[188.0, 305.0, 52.0, 101.0]","[2004.4549019607844, 652.2901960784313, 167.09...",0.977011,0.771258,58.526912,18.01877
113271,oiipfcng-1,14,0,0,0,0,0,14,6.276719,6.276719,...,1,0.758242,14.866069,"[4485.0, 850.0, 168.0, 1407.0]","[149.0, 124.0, 12.0, 111.0]","[1828.4855072463768, 487.4130434782609, 66.956...",0.926174,0.28261,42.384776,13.255454


In [13]:
obs_props_df_req.columns

Index(['cell_id', 'transcript_counts', 'control_probe_counts',
       'genomic_control_counts', 'control_codeword_counts',
       'unassigned_codeword_counts', 'deprecated_codeword_counts',
       'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count',
       'segmentation_method', 'region', 'z_level', 'cell_labels',
       'DAPI_PhenCyc', 'CD8_PhenCyc', 'CD31_PhenCyc', 'CD11c_PhenCyc',
       'CD34_PhenCyc', 'CD20_PhenCyc', 'CD4_PhenCyc', 'CD21_PhenCyc',
       'Ki67_PhenCyc', 'CD68_PhenCyc', 'E_cadherin_PhenCyc', 'pten_PhenCyc',
       'NF-H_PhenCyc', 'FOXp3_PhenCyc', 'CD45_PhenCyc', 'TFAM_PhenCyc',
       'PDL-1_PhenCyc', 'SOX2_PhenCyc', 'Vimentin_PhenCyc', 'HIF1A_PhenCyc',
       'CD44_PhenCyc', 'Keratin_8_18_PhenCyc', 'aSMA_PhenCyc', 'EpCAM_PhenCyc',
       'K14_PhenCyc', 'area', 'centroid-0', 'centroid-1', 'centroid_local-1',
       'centroid_weighted_local-0', 'centroid_weighted_local-1',
       'euler_number', 'extent', 'feret_diameter_max', 'intensity_max',
       'inten

In [ ]:
adata.obs = obs_props_df_req

import pandas as pd

# 1. Find all object-dtype columns in obs
obj_cols = [c for c, dt in adata.obs.dtypes.items() if dt == "object"]
print("Object-dtype columns:", obj_cols)

# 2. For each such column, inspect what types you actually have
for c in obj_cols:
    sample_types = set(type(v) for v in adata.obs[c].iloc[:100])
    print(f"  {c}: {sample_types}")



# 3b. Or, if you want to keep them, convert to string:
adata.obs[obj_cols] = adata.obs[obj_cols].astype(str)

# 4. (Optional) ensure index and column names are strings
adata.obs.index   = adata.obs.index.map(str)
adata.obs.columns = adata.obs.columns.map(str)


adata.write_h5ad("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_Region1_DownstreamAnalysis.annotationEnriched.image_props.phenocycler.h5ad")
sdata.tables["GeneTranscripts_XStock_PhenCyc_Props"] = adata
sdata.write("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.zarr")

/Users/ugursahin/miniforge3/envs/SOPA/lib/python3.10/site-packages/spatialdata/_core/_elements.py:125: UserWarning: Key `GeneTranscripts_XStock_PhenCyc_Props` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)


INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside /Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.zarr). Please see 
         the documentation of `is_self_contained()` to understand the implications of working with SpatialData     
         objects that are not self-contained.                                                                      
INFO     The Zarr backing store has been changed from                                                              
         /Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V5.zarr the new file path:             
         /Volumes/DATA_2025/Projects/RAO/dbs/Xenium_and_AdjustedPhenoCycler.V6.zarr                                


Object-dtype columns: ['cell_id', 'area', 'centroid-0', 'centroid-1', 'centroid_local-1', 'centroid_weighted_local-0', 'centroid_weighted_local-1', 'euler_number', 'extent', 'feret_diameter_max', 'intensity_max', 'intensity_min', 'intensity_mean', 'solidity', 'eccentricity', 'perimeter', 'equivalent_diameter_area']
  cell_id: {<class 'str'>}
  area: {<class 'numpy.float64'>}
  centroid-0: {<class 'numpy.float64'>}
  centroid-1: {<class 'numpy.float64'>}
  centroid_local-1: {<class 'numpy.float64'>}
  centroid_weighted_local-0: {<class 'list'>}
  centroid_weighted_local-1: {<class 'list'>}
  euler_number: {<class 'numpy.int64'>}
  extent: {<class 'numpy.float64'>}
  feret_diameter_max: {<class 'numpy.float64'>}
  intensity_max: {<class 'list'>}
  intensity_min: {<class 'list'>}
  intensity_mean: {<class 'list'>}
  solidity: {<class 'numpy.float64'>}
  eccentricity: {<class 'numpy.float64'>}
  perimeter: {<class 'numpy.float64'>}
  equivalent_diameter_area: {<class 'numpy.float64'>}


In [136]:
adata.write_h5ad("/Volumes/DATA_2025/Projects/RAO/dbs/Xenium_Region1_DownstreamAnalysis.annotationEnriched.image_props.phenocycler.h5ad")